In [1]:
import sys
import os

sys.path.append(os.path.abspath('bundled/tool'))

import unittest
from unittest.mock import MagicMock
import lsprotocol.types as lsp

from common.completion import get_completion_items
from common.symbols import get_doc_symbols, fill_workspace
from jaclang.jac.workspace import Workspace

In [5]:
from unittest.mock import MagicMock
from jaclang.jac.workspace import Workspace

class MockLanguageServer:
    def __init__(self, root_path, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.workspace = MockWorkspace(root_path)
        self.dep_table = {}

class MockWorkspace:
    def __init__(self, root_path):
        self.documents = {}
        self.root_path = root_path

    def put_document(self, doc):
        self.documents[doc.uri] = MockDocument(doc)

    def get_text_document(self, uri):
        return self.documents[uri]

    def get_document(self, uri):
        return self.get_text_document(uri)
    
class MockDocument:
    def __init__(self, doc):
        self.text = doc.text
        self.uri = doc.uri
        self.version = doc.version
        self.language_id = doc.language_id

In [6]:
ls = MockLanguageServer('bundled/tool/tests/fixtures/completion')

In [7]:
fill_workspace(ls)

In [20]:
ls.workspace.get_text_document('file://bundled/tool/tests/fixtures/completion/completions.jac').symbols[0].ws_symbol

Symbol(GuessGame, walker, public, None, [<jaclang.jac.absyntree.Architype object at 0x1126d4f50>])

In [2]:
len(':walker:GuessGame:ability:start')

31

In [13]:
ls.jlws = Workspace(ls.workspace.root_path)

In [9]:
from lsprotocol.types import TextDocumentItem

for mod_path, mod_info in ls.jlws.modules.items():
    print(mod_path)
    print(mod_info.ir.sym_tab)
    doc = TextDocumentItem(
        uri=f"file://{mod_path}",
        language_id="jac",
        version=0,
        text=mod_info.ir.source.code,
    )
    ls.workspace.put_document(doc)
    doc = ls.workspace.get_text_document(doc.uri)
    doc.symbols = get_doc_symbols(ls, doc.uri)

for doc in ls.workspace.docs.values():
    doc_url = doc.uri.replace("file://", "")
    doc.dependencies = {}
    jlws_imports = ls.jlws.get_dependencies(doc_url)

bundled/tool/tests/fixtures/completion/completions.jac
completions <jaclang.jac.symtable.SymbolTable object at 0x11355e2d0>:
    GuessGame: Symbol(GuessGame, walker, public, None, [<jaclang.jac.absyntree.Architype object at 0x113656cd0>])
    turn: Symbol(turn, node, public, None, [<jaclang.jac.absyntree.Architype object at 0x113668110>])
    (n)turn.(a)check: Symbol((n)turn.(a)check, impl, public, None, [<jaclang.jac.absyntree.AbilityDef object at 0x113669f10>])
    (w)GuessGame.(a)start_game: Symbol((w)GuessGame.(a)start_game, impl, public, None, [<jaclang.jac.absyntree.AbilityDef object at 0x1136700d0>])
    (w)GuessGame.(a)process_guess: Symbol((w)GuessGame.(a)process_guess, impl, public, None, [<jaclang.jac.absyntree.AbilityDef object at 0x113675ed0>])



In [59]:
doc = ls.workspace.get_text_document('file://examples/circle_impl.jac')
doc.symbols

In [15]:
ls_mock = MagicMock()
ls_mock.workspace = MockWorkspace()
ls_mock.jlws = Workspace(path="bundled/tool/tests/fixtures/completion/")

In [16]:
params = lsp.CompletionParams(
    text_document=lsp.TextDocumentIdentifier(
        uri="file://bundled/tool/tests/fixtures/completion/completions.jac"
    ),
    position=lsp.Position(line=15, character=0),
)

AttributeError: 'MockWorkspace' object has no attribute 'root_path'

In [18]:
doc = ls_mock.workspace.get_text_document(params.text_document.uri)

In [20]:
doc.symbols = get_doc_symbols(ls_mock, doc.uri)

In [41]:
doc.symbols

In [42]:
get_completion_items(ls_mock, params)

AttributeError: 'NoneType' object has no attribute 'decl'